In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GroupShuffleSplit
import wandb


csv_path = "../../data/csv/expanded_dataset_evidence.csv" 
# Load dataset
df = pd.read_csv(csv_path)  # Replace with your actual file path

# Train/test split
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state=0)
split = splitter.split(df, groups=df['Participant'])

train_inds, test_inds = next(split)

train = df.iloc[train_inds]
test = df.iloc[test_inds]

print("Train participants: ", set(train["Participant"]))
print("Test participants: ", set(test["Participant"]))


Train participants:  {2.0, 4.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 18.0, 20.0, 21.0, 23.0, 24.0, 29.0, 30.0, 32.0, 9003.0, 9004.0, 9005.0, 9008.0, 1010.0, 1012.0, 1013.0, 1014.0, 1017.0, 1018.0, 1019.0, 1020.0}
Test participants:  {33.0, 7.0, 9001.0, 9002.0, 19.0, 1011.0, 25.0, 28.0}


In [2]:
# Encode 'yes' as 1 and 'no' as 0
def encode_sequence(seq):
    return [1 if token.strip().lower() == "yes" else 0 for token in seq.split(',')]

# Encode all sequences
train_seq = train['evidence_sequence'].apply(encode_sequence)
test_seq = test['evidence_sequence'].apply(encode_sequence)

# Find max sequence length
max_len = max(len(seq) for seq in train_seq)

# Pad sequences with 0s (or trim if too long)
def pad_or_trim(seq, target_len):
    if len(seq) < target_len:
        return seq + [0] * (target_len - len(seq))  # pad with 0s
    else:
        return seq[:target_len]  # trim if too long

Xtrain = np.array([pad_or_trim(seq, max_len) for seq in train_seq])
y_train = train['experience'].values

Xtest = np.array([pad_or_trim(seq, max_len) for seq in test_seq])
y_test = test['experience'].values

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)


# Train SVM
clf = SVC(kernel='rbf', C=1.0, gamma='scale', decision_function_shape='ovo')
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[ 93   8  13  15]
 [146  28  22  17]
 [ 56  12   9  10]
 [156  35  37  21]]

Classification Report:
               precision    recall  f1-score   support

         CSI       0.21      0.72      0.32       129
     Control       0.34      0.13      0.19       213
   FirstYear       0.11      0.10      0.11        87
   ThirdYear       0.33      0.08      0.13       249

    accuracy                           0.22       678
   macro avg       0.25      0.26      0.19       678
weighted avg       0.28      0.22      0.18       678



In [3]:
print(len(X_train), len(X_test))

2253 678


In [4]:
train

,Participant,evidence_sequence,experience
0,2.0,"no,no,no,no,yes,no,no,no,yes,no",Control
1,2.0,"no,no,no,no,no,no,no,no,no,no",Control
2,2.0,"no,no,yes,yes,no,no,no,no,no,no",Control
3,2.0,"no,no,no,no,no,no,no,no,no,no",Control
4,2.0,"no,no,no,yes,yes,no,no,no,no,no",Control
...,...,...,...
2926,9008.0,"no,no,yes,yes,yes,yes,yes,yes,yes,yes",Control
2927,9008.0,"yes,yes,yes,no,yes,yes,yes,yes,yes,no",Control
2928,9008.0,"no,no,no,no,no,no,no,no,yes,yes",Control
2929,9008.0,"yes,yes,yes,yes,yes,yes,yes,yes,yes,no",Control


In [5]:
test

,Participant,evidence_sequence,experience
208,7.0,"no,no,no,yes,yes,yes,no,no,yes,yes",Control
209,7.0,"no,no,yes,no,no,yes,yes,no,no,no",Control
210,7.0,"no,no,no,no,no,yes,yes,yes,no,yes",Control
211,7.0,"no,no,no,no,no,no,no,no,yes,no",Control
212,7.0,"no,yes,no,yes,no,no,no,no,yes,no",Control
...,...,...,...
2615,9002.0,"yes,yes,yes,yes,yes,yes,no,yes,yes,yes",Control
2616,9002.0,"yes,yes,yes,yes,yes,yes,yes,yes,yes,yes",Control
2617,9002.0,"yes,yes,yes,yes,yes,yes,yes,yes,yes,yes",Control
2618,9002.0,"yes,yes,yes,yes,no,no,yes,no,no,no",Control


In [6]:
import wandb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Initialize wandb
wandb.init(
    project="svm-random-search",
    name="svc_hyperparam_tuning",
    config={
        "model": "SVC",
        "search_type": "RandomizedSearchCV",
        "param_dist": {
            'C': [0.01, 0.1, 1, 10, 100, 1000],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'gamma': ['scale', 'auto', 1, 0.1, 0.01, 0.001, 0.0001],
            'degree': [2, 3, 4, 5],
            'coef0': [0.0, 0.1, 0.5, 1.0],
            'shrinking': [True, False],
            'decision_function_shape': ['ovo', 'ovr'],
            'class_weight': [None, 'balanced']
        },
        "n_iter": 100,
        "cv": 5,
        "scoring": "accuracy"
    }
)

# Define parameter space
param_dist = wandb.config["param_dist"]

# Initialize and fit RandomizedSearchCV
svc = SVC()
random_search = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_dist,
    n_iter=wandb.config["n_iter"],
    scoring=wandb.config["scoring"],
    cv=wandb.config["cv"],
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

# Log best parameters
wandb.log({"best_params": random_search.best_params_})

# Predict and evaluate
best_svc = random_search.best_estimator_
y_pred = best_svc.predict(X_test)

# Log confusion matrix, accuracy, and classification report
accuracy = accuracy_score(y_test, y_pred)
wandb.log({
    "test_accuracy": accuracy
})

# Also print to console

# Finish wandb run
wandb.finish()


wandb: Currently logged in as: sam-michiels (sam-michiels-open-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Fitting 5 folds for each of 100 candidates, totalling 500 fits


test_accuracy,▁
test_accuracy,0.19322


In [7]:
print("Best from Random Search:", random_search.best_params_)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Best from Random Search: {'shrinking': False, 'kernel': 'poly', 'gamma': 'scale', 'degree': 4, 'decision_function_shape': 'ovr', 'coef0': 1.0, 'class_weight': None, 'C': 0.01}
Confusion Matrix:
 [[112   2  14   1]
 [182   7  23   1]
 [ 69   6  10   2]
 [205  11  31   2]]

Classification Report:
               precision    recall  f1-score   support

         CSI       0.20      0.87      0.32       129
     Control       0.27      0.03      0.06       213
   FirstYear       0.13      0.11      0.12        87
   ThirdYear       0.33      0.01      0.02       249

    accuracy                           0.19       678
   macro avg       0.23      0.26      0.13       678
weighted avg       0.26      0.19      0.10       678



In [8]:
import joblib
joblib.dump(best_svc, 'svm_random_search_model_evidence.pkl')

['svm_random_search_model_evidence.pkl']